In [2]:
import ee
ee.Initialize()

In [3]:
import datetime
import pandas as pd

In [ ]:
#Extra comment

In [29]:
#def get_data(startTime,endTime,lat,lon):   
def get_data():
    startTime = datetime.datetime(2014, 1, 1)
    endTime = datetime.datetime(2014, 1, 20)
    
    point = {'type':'Point', 'coordinates':[ -122.144622,37.414345]};
    #try:
    if(True):
#        point = {'type':'Point', 'coordinates':[ -122.144622,37.414345]}; 
#        point = {'type':'Point', 'coordinates':[lon,lat]}; 
    
        #Read Landsat 8 Image
        L8_DN = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate(startTime, endTime)
        L8_DN = L8_DN.filterBounds(point)
        L8_DN = L8_DN.sort('CLOUD_COVER').first()

        image = ee.Image(L8_DN)
        tir = image.select(u'B10')
        red = image.select(u'B4')
        nir = image.select(u'B5')
        ndvi = nir.subtract(red).divide(nir.add(red))

        emissivity = ee.Image('NASA/ASTER_GED/AG100_003').select('emissivity_band10')
        srtm = ee.Image('USGS/SRTMGL1_003')

        
        
        # computes emissivity-corrected land surface temperature given Landsat8 band 10 (tir) and emissivity
        # assumes tir, emissivity are google earth engine images
        k1 = 774.8853 ; # constants for Landsat8 B10
        k2 =  1321.0789 ; # constants for Landsat8 B10
        dummyk1 = emissivity.multiply(0).add(1).multiply(k1) # make dummy variable same size/type as emissivity
        dummyk2 = emissivity.multiply(0).add(1).multiply(k2) # make dummy variable same size/type as emissivity
    
        # first convert band 10 Digital Number (DN) data to TOA spectral radiance
        radiance = tir.multiply(3.3420E-04).add(.1)  # scale by gain and bias, band specific 
        L_lamda = radiance.divide(emissivity); # correct for emissivity
        k1_over_L_lamda = dummyk1.divide(L_lamda) 
        denominator = k1_over_L_lamda.add(1).log(); 
        t = dummyk2.divide(denominator).subtract(273.15); # finally convert to degrees C 
    
    
        image = ee.ImageCollection(image)
        t = ee.ImageCollection(t)
        ndvi = ee.ImageCollection(ndvi)
        srtm = ee.ImageCollection(srtm)
    
        #Get the given Lat,Lon,date
        info = image.getRegion(point,500).getInfo()
        df_image = pd.DataFrame(info[1:],columns=info[0])
    
        #Get the LST (B10)
        info = t.getRegion(point,500).getInfo()
        df_t = pd.DataFrame(info[1:],columns=info[0])
        df_t.head()
    
        #Get the NDVI
        info = ndvi.getRegion(point,500).getInfo()
        df_ndvi = pd.DataFrame(info[1:],columns=info[0])

        #Get the SRTM
        info = srtm.getRegion(point,500).getInfo()
        df_srtm = pd.DataFrame(info[1:],columns=info[0])
    
#        df = pd.DataFrame()
#        df = pd.concat([df,df_image[['longitude','latitude','time']]],axis=1)
#        df = pd.concat([df,df_t],axis=1)
#        df = pd.concat([df,df_ndvi['B5']],axis=1)
#        df = pd.concat([df,df_srtm['elevation']],axis=1)
        
        #df.rename(index=str, columns={"B10": "LST"},inplace=True)
#        df.rename(index=str, columns={"B5": "NDVI"},inplace=True)
#        df['time'] = df['time'].apply(lambda x:datetime.datetime.fromtimestamp(float(str(x)[:-3])))
        return df_t
    #except:
    #    print("Not Found!")
    #    return pd.DataFrame

In [32]:
df = get_data()
df

,id,longitude,latitude,time,emissivity_band10
0,0,-122.146175,37.412586,None,-157.775435


In [30]:
import datetime
from matplotlib import dates
import matplotlib.dates as mdates
from pylab import *

startTime = datetime.datetime(2013, 1, 1)
endTime = datetime.datetime(2016, 1, 1)

collection = ee.ImageCollection('LANDSAT/LC8_L1T').filterDate(startTime, endTime)
point = {'type':'Point', 'coordinates':[ -122.055,37.9917]};  # LA
info = collection.getRegion(point,500).getInfo()

# extract the header column names
header = info[0]
# create a Numpy array of the data
data = array(info[1:])
# extract the time information
iTime = header.index('time')
# convert to Python datetime objects
time_collection = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

In [20]:
#Read list of metereological station
met_loc = pd.read_csv('met_locs_la.csv')
lat = met_loc['latitude'].tolist()
lon = met_loc['longitude'].tolist()
latlon = zip(lat,lon)

In [15]:
len(time_collection)

56

In [28]:
#Create the final Dataframe
df = pd.DataFrame()
counter=0

for time_X in time_collection[0:2]:
    print(counter)
    loc_counter = 0
    for loc in latlon[0:2]:
        #print(loc_counter)
        print(loc[0]) #lat
        print(loc[1]) #lon
        startTime = time_X-datetime.timedelta(days=1)
        endTime = time_X+datetime.timedelta(days=1)
        df_temp = get_data(startTime,endTime,loc[0],loc[1])
        try:
            df = pd.concat([df,df_temp])
        except:
            df = df.copy()
        loc_counter += 1
    counter += 1

0
37.9917
-122.055


ValueError: ('invalid literal for float(): 0    1366138073557\nName: time, dtype: in', u'occurred at index time')

In [11]:
df

""


In [ ]:
met_loc.head()